## Importación de librerías

In [1379]:
import pandas as pd

## Carga de datos

In [1380]:
df = pd.read_csv('../data/raw/bank-additional.csv')

In [1381]:
df.head()

,Unnamed: 0,age,job,marital,education,default,housing,loan,contact,duration,...,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,date,latitude,longitude,id_
0,0,NaN,housemaid,MARRIED,basic.4y,0.0,0.0,0.0,telephone,261,...,1.1,"93,994","-36,4","4,857",5191,no,2-agosto-2019,41.495,-71.233,089b39d8-e4d0-461b-87d4-814d71e0e079
1,1,57.0,services,MARRIED,high.school,NaN,0.0,0.0,telephone,149,...,1.1,"93,994","-36,4",NaN,5191,no,14-septiembre-2016,34.601,-83.923,e9d37224-cb6f-4942-98d7-46672963d097
2,2,37.0,services,MARRIED,high.school,0.0,1.0,0.0,telephone,226,...,1.1,"93,994","-36,4","4,857",5191,no,15-febrero-2019,34.939,-94.847,3f9f49b5-e410-4948-bf6e-f9244f04918b
3,3,40.0,admin.,MARRIED,basic.6y,0.0,0.0,0.0,telephone,151,...,1.1,"93,994","-36,4",NaN,5191,no,29-noviembre-2015,49.041,-70.308,9991fafb-4447-451a-8be2-b0df6098d13e
4,4,56.0,services,MARRIED,high.school,0.0,0.0,1.0,telephone,307,...,1.1,"93,994","-36,4",NaN,5191,no,29-enero-2017,38.033,-104.463,eca60b76-70b6-4077-80ba-bc52e8ebb0eb


## Selección de columnas

Vamos a eliminar columnas que no nos aportan información relevante:

* `Unnamed: 0`: es un índice antiguo que se guardó por error al exportar el CSV. No tiene valor analítico y puede eliminarse sin problema.

* `date`: Aunque tiene una gran cantidad de valores distintos (1.825 fechas únicas), no parece aportar información útil para agrupar o detectar patrones temporales, y además contiene valores nulos. Podríamos descomponerla en componentes si quisiéramos hacer análisis temporal, pero en este caso decidimos eliminarla.

* `default`: El 100% de los valores (o casi todos) son 0, por lo tanto no aporta variación ni valor predictivo. La eliminamos.

* `contact_year`: Todas las campañas suceden en un mismo rango de tiempo (2010–2013). No hay suficiente variación para aportar valor. Podríamos usarlo para análisis temporal, pero no será el enfoque. Se elimina. No sale en la base de datos.

* `contact_month`: Aunque tiene sentido como variable temporal, si no se va a hacer análisis de estacionalidad o descomposición por mes, podemos eliminarla. Además, los valores categóricos pueden estar desbalanceados. No sale en la base de datos.

* `pdays`: El 99% de los valores son 999, que indica "no contactado anteriormente". Este valor constante hace que la variable sea poco útil tal cual está. Si no la transformamos binariamente ("fue contactado antes o no"), podemos eliminarla.

* `previous`: Tiene un comportamiento similar a pdays, ya que la mayoría de los registros son 0 (nunca contactado antes). Si no la transformamos, podemos eliminarla por su bajo aporte.

* `latitude` y `longitude` las eliminamos porque no tenemos suficiente contexto geográfico para interpretarlas ni están relacionadas directamente con el objetivo del análisis, por lo que no aportan valor en este caso.

## Eliminar duplicados

In [1382]:
df = df.drop_duplicates()

In [1383]:
columnas_eliminar = [
    'Unnamed: 0',
    'date',
    'default',
    'latitude',
    'longitude',
    'pdays',
    'previous'
    ]

In [1384]:
df.drop(columns=columnas_eliminar)

,age,job,marital,education,housing,loan,contact,duration,campaign,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,id_
0,NaN,housemaid,MARRIED,basic.4y,0.0,0.0,telephone,261,1,NONEXISTENT,1.1,"93,994","-36,4","4,857",5191,no,089b39d8-e4d0-461b-87d4-814d71e0e079
1,57.0,services,MARRIED,high.school,0.0,0.0,telephone,149,1,NONEXISTENT,1.1,"93,994","-36,4",NaN,5191,no,e9d37224-cb6f-4942-98d7-46672963d097
2,37.0,services,MARRIED,high.school,1.0,0.0,telephone,226,1,NONEXISTENT,1.1,"93,994","-36,4","4,857",5191,no,3f9f49b5-e410-4948-bf6e-f9244f04918b
3,40.0,admin.,MARRIED,basic.6y,0.0,0.0,telephone,151,1,NONEXISTENT,1.1,"93,994","-36,4",NaN,5191,no,9991fafb-4447-451a-8be2-b0df6098d13e
4,56.0,services,MARRIED,high.school,0.0,1.0,telephone,307,1,NONEXISTENT,1.1,"93,994","-36,4",NaN,5191,no,eca60b76-70b6-4077-80ba-bc52e8ebb0eb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42995,NaN,admin.,MARRIED,university.degree,0.0,0.0,cellular,618,2,NONEXISTENT,1.4,"93,444","-36,1",NaN,"5228,1",yes,4eed05de-2a98-4227-b488-32122009b638
42996,34.0,technician,MARRIED,professional.course,1.0,1.0,cellular,42,7,NONEXISTENT,-0.1,"93,2",-42,NaN,"5195,8",no,0f0aca88-4088-4fe2-905f-44fb675d9493
42997,NaN,blue-collar,SINGLE,basic.6y,1.0,0.0,cellular,391,2,NONEXISTENT,1.4,"93,918","-42,7",NaN,"5228,1",no,cadadd4b-7ee5-4019-b13a-ca01bb67ca5b
42998,NaN,admin.,MARRIED,university.degree,0.0,0.0,cellular,674,3,NONEXISTENT,1.4,"93,918","-42,7","4,958","5228,1",no,5f432048-d515-4bb5-9c94-62db451f88d4


In [1385]:
df = df.drop(columns=columnas_eliminar)

In [1386]:
df.duplicated().sum()

np.int64(0)

In [1387]:
df.duplicated().mean()

np.float64(0.0)

Fijémonos en que ahora puede haber duplicados, ya que podrían ser distintos clientes con características similares. En este caso, no tenemos forma de saber si se trata del mismo cliente o no, ya que no contamos con un identificador único `id_`, así que los vamos a dejar por si acaso. No borramos esta variable al final.

## Limpieza de datos

Vamos a hacer una limpieza columna a columna.

In [1388]:
df.columns

Index(['age', 'job', 'marital', 'education', 'housing', 'loan', 'contact',
       'duration', 'campaign', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y', 'id_'],
      dtype='object')

### Columna `age`

In [1389]:
print(f"El número de valores nulos en la columna age es: {df['age'].isna().sum()}")

El número de valores nulos en la columna age es: 5120


In [1390]:
df['age']

0         NaN
1        57.0
2        37.0
3        40.0
4        56.0
         ... 
42995     NaN
42996    34.0
42997     NaN
42998     NaN
42999     NaN
Name: age, Length: 43000, dtype: float64

Como teníamos un 10% de nulos en la columna `age`, no eliminamos los nulos y rellenamos con la media.

In [1391]:
df['age'] = df['age'].fillna(df['age'].mean())

Nos aseguramos que no quedan nulos.

In [1392]:
print(df['age'].isnull().sum())


0


Vamos a revisar si tenemos outliers.

In [1393]:
print(df['age'].describe())


count    43000.000000
mean        39.977112
std          9.796830
min         17.000000
25%         33.000000
50%         39.977112
75%         46.000000
max         98.000000
Name: age, dtype: float64


Vamos a eliminar las edades que no son razonables, por ejemplo, menores de 18 años o mayores de 90 años.

In [1394]:
df = df[(df['age'] >= 18) & (df['age'] < 90)] # Esto asegura que solo se queden las personas que tienen entre 18 y 89 años (inclusive)


In [1395]:
print(df['age'].describe())

count    42985.000000
mean        39.967333
std          9.761246
min         18.000000
25%         33.000000
50%         39.977112
75%         46.000000
max         89.000000
Name: age, dtype: float64


Quedaría así la columna de `age`.

### Columna `job`

In [1396]:
print(f"El número de valores nulos en la columna job es: {df['job'].isna().sum()}")

El número de valores nulos en la columna job es: 345


In [1397]:
df['job']

0          housemaid
1           services
2           services
3             admin.
4           services
            ...     
42995         admin.
42996     technician
42997    blue-collar
42998         admin.
42999     unemployed
Name: job, Length: 42985, dtype: object

In [1398]:
print(df['job'].describe())

count      42640
unique        11
top       admin.
freq       10873
Name: job, dtype: object


In [1399]:
print(df['job'].value_counts(dropna=False))


job
admin.           10873
blue-collar       9654
technician        7026
services          4162
management        3050
retired           1780
entrepreneur      1522
self-employed     1489
housemaid         1123
unemployed        1063
student            898
NaN                345
Name: count, dtype: int64


Como los nulos solo representan el 0.8% del total, y estamos en fase de limpieza para análisis descriptivo, vamos a rellenarlos con la moda.

In [1400]:
(df['job'].mode()[0])

'admin.'

In [1401]:
df['job'] = df['job'].fillna(df['job'].mode()[0])


In [1402]:
print(df['job'].value_counts(dropna=False))


job
admin.           11218
blue-collar       9654
technician        7026
services          4162
management        3050
retired           1780
entrepreneur      1522
self-employed     1489
housemaid         1123
unemployed        1063
student            898
Name: count, dtype: int64


Quedaría así el total de cada profesión.

### Columna `marital`

In [1403]:
print(f"El número de valores nulos en la columna marital es: {df['marital'].isna().sum()}")

El número de valores nulos en la columna marital es: 85


In [1404]:
df['marital']

0        MARRIED
1        MARRIED
2        MARRIED
3        MARRIED
4        MARRIED
          ...   
42995    MARRIED
42996    MARRIED
42997     SINGLE
42998    MARRIED
42999     SINGLE
Name: marital, Length: 42985, dtype: object

In [1405]:
print(df['marital'].describe())

count       42900
unique          3
top       MARRIED
freq        25991
Name: marital, dtype: object


In [1406]:
print(df['marital'].value_counts(dropna=False))

marital
MARRIED     25991
SINGLE      12100
DIVORCED     4809
NaN            85
Name: count, dtype: int64


Como los nulos solo representan el 0.2% del total, y estamos en fase de limpieza para análisis descriptivo, vamos a rellenarlos con la moda.

In [1407]:
(df['marital'].mode()[0])

'MARRIED'

In [1408]:
df['marital'] = df['marital'].fillna(df['marital'].mode()[0])

In [1409]:
print(df['marital'].value_counts(dropna=False))

marital
MARRIED     26076
SINGLE      12100
DIVORCED     4809
Name: count, dtype: int64


Quedaría así el total del estado civil del cliente.

### Columna `education`

In [1410]:
print(f"El número de valores nulos en la columna education es: {df['education'].isna().sum()}")

El número de valores nulos en la columna education es: 1801


In [1411]:
df['education']

0                   basic.4y
1                high.school
2                high.school
3                   basic.6y
4                high.school
                ...         
42995      university.degree
42996    professional.course
42997               basic.6y
42998      university.degree
42999      university.degree
Name: education, Length: 42985, dtype: object

In [1412]:
print(df['education'].describe())

count                 41184
unique                    7
top       university.degree
freq                  12720
Name: education, dtype: object


In [1413]:
print(df['education'].value_counts(dropna=False))

education
university.degree      12720
high.school             9925
basic.9y                6305
professional.course     5477
basic.4y                4354
basic.6y                2385
NaN                     1801
illiterate                18
Name: count, dtype: int64


Como los nulos solo representan el 4.2% del total, y estamos en fase de limpieza para análisis descriptivo, vamos a rellenarlos con la moda.

In [1414]:
(df['education'].mode()[0])

'university.degree'

In [1415]:
df['education'] = df['education'].fillna(df['education'].mode()[0])

In [1416]:
print(df['education'].value_counts(dropna=False))

education
university.degree      14521
high.school             9925
basic.9y                6305
professional.course     5477
basic.4y                4354
basic.6y                2385
illiterate                18
Name: count, dtype: int64


Quedaría así el total del nivel de educación del cliente.

### Columna `housing`

In [1417]:
print(f"El número de valores nulos en la columna housing es: {df['housing'].isna().sum()}")

El número de valores nulos en la columna housing es: 1025


In [1418]:
df['housing']

0        0.0
1        0.0
2        1.0
3        0.0
4        0.0
        ... 
42995    0.0
42996    1.0
42997    1.0
42998    0.0
42999    0.0
Name: housing, Length: 42985, dtype: float64

In [1419]:
print(df['housing'].describe())

count    41960.000000
mean         0.536010
std          0.498708
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: housing, dtype: float64


In [1420]:
print(df['housing'].value_counts(dropna=False))

housing
1.0    22491
0.0    19469
NaN     1025
Name: count, dtype: int64


Como los nulos solo representan el 2.39% del total, y estamos en fase de limpieza para análisis descriptivo, vamos a rellenarlos con la moda.

In [1421]:
(df['housing'].mode()[0])

np.float64(1.0)

In [1422]:
df['housing'] = df['housing'].fillna(df['housing'].mode()[0])

In [1423]:
print(df['housing'].value_counts(dropna=False))

housing
1.0    23516
0.0    19469
Name: count, dtype: int64


Quedarían así los valores de la columna `housing` que nos indica si el cliente tiene un préstamos hipotecario o no.

### Columna `loan`

In [1424]:
print(f"El número de valores nulos en la columna loan es: {df['loan'].isna().sum()}")

El número de valores nulos en la columna loan es: 1025


In [1425]:
df['loan']

0        0.0
1        0.0
2        0.0
3        0.0
4        1.0
        ... 
42995    0.0
42996    1.0
42997    0.0
42998    0.0
42999    1.0
Name: loan, Length: 42985, dtype: float64

In [1426]:
print(df['loan'].describe())

count    41960.000000
mean         0.155553
std          0.362435
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: loan, dtype: float64


In [1427]:
print(df['loan'].value_counts(dropna=False))

loan
0.0    35433
1.0     6527
NaN     1025
Name: count, dtype: int64


Como los nulos solo representan el 2.39% del total, y estamos en fase de limpieza para análisis descriptivo, vamos a rellenarlos con la moda.

In [1428]:
(df['loan'].mode()[0])

np.float64(0.0)

In [1429]:
df['loan'] = df['loan'].fillna(df['loan'].mode()[0])

In [1430]:
print(df['loan'].value_counts(dropna=False))

loan
0.0    36458
1.0     6527
Name: count, dtype: int64


Quedarían así los valores de la columna `loan` que nos indica si el cliente tiene algún otro tipo de préstamo.

### Columna `contact`

In [1431]:
print(f"El número de valores nulos en la columna contact es: {df['contact'].isna().sum()}")

El número de valores nulos en la columna contact es: 0


In [1432]:
df['contact']

0        telephone
1        telephone
2        telephone
3        telephone
4        telephone
           ...    
42995     cellular
42996     cellular
42997     cellular
42998     cellular
42999     cellular
Name: contact, Length: 42985, dtype: object

In [1433]:
print(df['contact'].describe())

count        42985
unique           2
top       cellular
freq         27381
Name: contact, dtype: object


In [1434]:
print(df['contact'].value_counts(dropna=False))

contact
cellular     27381
telephone    15604
Name: count, dtype: int64


Como no hay nulos, esta columna está lista para el análisis descriptivo. Quedarían así los valores de la columna `contact` que nos indica el método de contacto utilizado para comunicarse con el cliente.

### Columna `duration`

In [1435]:
print(f"El número de valores nulos en la columna duration es: {df['duration'].isna().sum()}")

El número de valores nulos en la columna duration es: 0


In [1436]:
df['duration']

0        261
1        149
2        226
3        151
4        307
        ... 
42995    618
42996     42
42997    391
42998    674
42999    104
Name: duration, Length: 42985, dtype: int64

In [1437]:
print(df['duration'].describe())

count    42985.000000
mean       257.684122
std        258.637276
min          0.000000
25%        102.000000
50%        179.000000
75%        319.000000
max       4918.000000
Name: duration, dtype: float64


In [1438]:
print(df['duration'].value_counts(dropna=False))

duration
90      183
136     177
85      176
73      172
111     170
       ... 
1416      1
1603      1
1361      1
2219      1
1048      1
Name: count, Length: 1540, dtype: int64


Como no hay nulos, esta columna está lista para el análisis descriptivo. Quedarían así los valores de la columna `duration` que nos indica la duración en segundos de la última interacción con el cliente.

### Columna `campaign`

In [1439]:
print(f"El número de valores nulos en la columna campaign es: {df['campaign'].isna().sum()}")

El número de valores nulos en la columna campaign es: 0


In [1440]:
df['campaign']

0        1
1        1
2        1
3        1
4        1
        ..
42995    2
42996    7
42997    2
42998    3
42999    2
Name: campaign, Length: 42985, dtype: int64

In [1441]:
print(df['campaign'].describe())

count    42985.000000
mean         2.567547
std          2.772687
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         56.000000
Name: campaign, dtype: float64


In [1442]:
print(df['campaign'].value_counts(dropna=False))

campaign
1     18396
2     11044
3      5581
4      2777
5      1658
6      1025
7       658
8       418
9       289
10      236
11      183
12      130
13       94
14       73
17       61
16       51
15       51
18       34
20       30
19       26
21       26
23       20
24       18
22       17
27       11
29       10
30       10
28        8
31        8
25        8
26        8
35        5
33        4
32        4
34        3
43        2
40        2
42        2
39        1
56        1
37        1
41        1
Name: count, dtype: int64


Como no hay nulos, esta columna está lista para el análisis descriptivo. Quedarían así los valores de la columna `campaign` que nos indica el número de contactos realizados durante esta campaña para este cliente.

### Columna `poutcome`

In [1443]:
print(f"El número de valores nulos en la columna poutcome es: {df['poutcome'].isna().sum()}")

El número de valores nulos en la columna poutcome es: 0


In [1444]:
df['poutcome']

0        NONEXISTENT
1        NONEXISTENT
2        NONEXISTENT
3        NONEXISTENT
4        NONEXISTENT
            ...     
42995    NONEXISTENT
42996    NONEXISTENT
42997    NONEXISTENT
42998    NONEXISTENT
42999    NONEXISTENT
Name: poutcome, Length: 42985, dtype: object

In [1445]:
print(df['poutcome'].describe())

count           42985
unique              3
top       NONEXISTENT
freq            37100
Name: poutcome, dtype: object


In [1446]:
print(df['poutcome'].value_counts(dropna=False))

poutcome
NONEXISTENT    37100
FAILURE         4456
SUCCESS         1429
Name: count, dtype: int64


Como no hay nulos, esta columna está lista para el análisis descriptivo. Quedarían así los valores de la columna `poutcome` que nos indica el resultado de la campaña de marketing anterior.

### Columna `emp.var.rate`

In [1447]:
print(f"El número de valores nulos en la columna emp.var.rate es: {df['emp.var.rate'].isna().sum()}")

El número de valores nulos en la columna emp.var.rate es: 0


In [1448]:
df['emp.var.rate']

0        1.1
1        1.1
2        1.1
3        1.1
4        1.1
        ... 
42995    1.4
42996   -0.1
42997    1.4
42998    1.4
42999   -0.1
Name: emp.var.rate, Length: 42985, dtype: float64

In [1449]:
print(df['emp.var.rate'].describe())

count    42985.000000
mean         0.078104
std          1.573250
min         -3.400000
25%         -1.800000
50%          1.100000
75%          1.400000
max          1.400000
Name: emp.var.rate, dtype: float64


In [1450]:
print(df['emp.var.rate'].value_counts(dropna=False))

emp.var.rate
 1.4    16980
-1.8     9617
 1.1     8020
-0.1     3830
-2.9     1744
-3.4     1127
-1.7      810
-1.1      665
-3.0      181
-0.2       11
Name: count, dtype: int64


Como no hay nulos, esta columna está lista para el análisis descriptivo. Quedarían así los valores de la columna `emp.var.rate` que nos indica la tasa de variación del empleo.

### Columna `cons.price.idx`

In [1451]:
print(f"El número de valores nulos en la columna cons.price.idx es: {df['cons.price.idx'].isna().sum()}")

El número de valores nulos en la columna cons.price.idx es: 471


In [1452]:
df['cons.price.idx']

0        93,994
1        93,994
2        93,994
3        93,994
4        93,994
          ...  
42995    93,444
42996      93,2
42997    93,918
42998    93,918
42999      93,2
Name: cons.price.idx, Length: 42985, dtype: object

In [1453]:
print(df['cons.price.idx'].describe())

count      42514
unique        26
top       93,994
freq        7938
Name: cons.price.idx, dtype: object


In [1454]:
print(df['cons.price.idx'].value_counts(dropna=False))

cons.price.idx
93,994    7938
93,918    6937
92,893    5985
93,444    5349
94,465    4522
93,2      3731
93,075    2552
92,201     799
92,963     742
NaN        471
92,431     468
92,649     374
94,215     320
94,199     316
92,843     297
92,379     272
93,369     271
94,027     240
94,055     236
93,876     222
94,601     212
92,469     184
92,713     179
93,749     179
94,767     130
93,798      48
92,756      11
Name: count, dtype: int64


Como los nulos solo representan el 1.10% del total, y estamos en fase de limpieza para análisis descriptivo, vamos a rellenarlos con la moda.

In [1455]:
(df['cons.price.idx'].mode()[0])

'93,994'

In [1456]:
df['cons.price.idx'] = df['cons.price.idx'].fillna(df['cons.price.idx'].mode()[0])

In [1457]:
print(df['cons.price.idx'].value_counts(dropna=False))

cons.price.idx
93,994    8409
93,918    6937
92,893    5985
93,444    5349
94,465    4522
93,2      3731
93,075    2552
92,201     799
92,963     742
92,431     468
92,649     374
94,215     320
94,199     316
92,843     297
92,379     272
93,369     271
94,027     240
94,055     236
93,876     222
94,601     212
92,469     184
92,713     179
93,749     179
94,767     130
93,798      48
92,756      11
Name: count, dtype: int64


Quedarían así los valores de la columna `cons.price.idx` que nos indica el índice de precios al consumidor.

### Columna `cons.conf.idx`

In [1458]:
print(f"El número de valores nulos en la columna cons.conf.idx es: {df['cons.conf.idx'].isna().sum()}")

El número de valores nulos en la columna cons.conf.idx es: 0


In [1459]:
df['cons.conf.idx']

0        -36,4
1        -36,4
2        -36,4
3        -36,4
4        -36,4
         ...  
42995    -36,1
42996      -42
42997    -42,7
42998    -42,7
42999      -42
Name: cons.conf.idx, Length: 42985, dtype: object

In [1460]:
print(df['cons.conf.idx'].describe())

count     42985
unique       26
top       -36,4
freq       8020
Name: cons.conf.idx, dtype: object


In [1461]:
print(df['cons.conf.idx'].value_counts(dropna=False))

cons.conf.idx
-36,4    8020
-42,7    7004
-46,2    6057
-36,1    5408
-41,8    4568
-42      3782
-47,1    2581
-31,4     811
-40,8     748
-26,9     474
-30,1     378
-40,3     328
-37,5     318
-50       298
-34,8     275
-29,8     275
-38,3     243
-39,8     239
-40       223
-49,5     214
-33,6     185
-34,6     183
-33       181
-50,8     133
-40,4      48
-45,9      11
Name: count, dtype: int64


Como no hay nulos, esta columna está lista para el análisis descriptivo. Quedarían así los valores de la columna `cons.conf.idx` que nos indica el índice de confianza del consumidor.

### Columna `euribor3m`

In [1462]:
print(f"El número de valores nulos en la columna euribor3m es: {df['euribor3m'].isna().sum()}")

El número de valores nulos en la columna euribor3m es: 9252


In [1463]:
df['euribor3m']

0        4,857
1          NaN
2        4,857
3          NaN
4          NaN
         ...  
42995      NaN
42996      NaN
42997      NaN
42998    4,958
42999    4,021
Name: euribor3m, Length: 42985, dtype: object

In [1464]:
print(df['euribor3m'].describe())

count     33733
unique      309
top       4,857
freq       2287
Name: euribor3m, dtype: object


In [1465]:
print(df['euribor3m'].value_counts(dropna=False))

euribor3m
NaN      9252
4,857    2287
4,962    2124
4,963    2019
4,961    1594
         ... 
5,045       1
4,921       1
0,956       1
0,894       1
0,953       1
Name: count, Length: 310, dtype: int64


Como los nulos representan el 21.53% del total, y estamos en fase de limpieza para análisis descriptivo, vamos a rellenarlos con creando una nueva categoría que se llame `unknown`.

In [1466]:
df['euribor3m'] = df['euribor3m'].fillna('unknown')

In [1467]:
df['euribor3m'].value_counts()

euribor3m
unknown    9252
4,857      2287
4,962      2124
4,963      2019
4,961      1594
           ... 
5,045         1
4,921         1
0,956         1
0,894         1
0,953         1
Name: count, Length: 310, dtype: int64

Esta columna está lista para el análisis descriptivo. Quedarían así los valores de la columna `euribor3m` que nos indica la tasa de interés de referencia a tres meses.

### Columna `nr.employed`

In [1468]:
print(f"El número de valores nulos en la columna nr.employed es: {df['nr.employed'].isna().sum()}")

El número de valores nulos en la columna nr.employed es: 0


In [1469]:
df['nr.employed']

0          5191
1          5191
2          5191
3          5191
4          5191
          ...  
42995    5228,1
42996    5195,8
42997    5228,1
42998    5228,1
42999    5195,8
Name: nr.employed, Length: 42985, dtype: object

In [1470]:
print(df['nr.employed'].describe())

count      42985
unique        11
top       5228,1
freq       16980
Name: nr.employed, dtype: object


In [1471]:
print(df['nr.employed'].value_counts(dropna=False))

nr.employed
5228,1    16980
5099,1     8936
5191       8020
5195,8     3830
5076,2     1744
5017,5     1127
4991,6      810
5008,7      681
4963,6      665
5023,5      181
5176,3       11
Name: count, dtype: int64


Como no hay nulos, esta columna está lista para el análisis descriptivo. Quedarían así los valores de la columna `nr.employed` que nos indica el número de empleados.

### Columna `y`

In [1472]:
print(f"El número de valores nulos en la columna y es: {df['y'].isna().sum()}")

El número de valores nulos en la columna y es: 0


In [1473]:
df['y']

0         no
1         no
2         no
3         no
4         no
        ... 
42995    yes
42996     no
42997     no
42998     no
42999     no
Name: y, Length: 42985, dtype: object

In [1474]:
print(df['y'].describe())

count     42985
unique        2
top          no
freq      38148
Name: y, dtype: object


In [1475]:
print(df['y'].value_counts(dropna=False))

y
no     38148
yes     4837
Name: count, dtype: int64


Como no hay nulos, esta columna está lista para el análisis descriptivo. Quedarían así los valores de la columna `y` que nos indica si el cliente ha suscrito un producto o servicio.

### Columna `id_`

In [1476]:
print(f"El número de valores nulos en la columna id_ es: {df['id_'].isna().sum()}")

El número de valores nulos en la columna id_ es: 0


In [1477]:
df['id_']

0        089b39d8-e4d0-461b-87d4-814d71e0e079
1        e9d37224-cb6f-4942-98d7-46672963d097
2        3f9f49b5-e410-4948-bf6e-f9244f04918b
3        9991fafb-4447-451a-8be2-b0df6098d13e
4        eca60b76-70b6-4077-80ba-bc52e8ebb0eb
                         ...                 
42995    4eed05de-2a98-4227-b488-32122009b638
42996    0f0aca88-4088-4fe2-905f-44fb675d9493
42997    cadadd4b-7ee5-4019-b13a-ca01bb67ca5b
42998    5f432048-d515-4bb5-9c94-62db451f88d4
42999    993bbbd6-4dbc-4a40-a408-f91f8462bee6
Name: id_, Length: 42985, dtype: object

In [1478]:
print(df['id_'].describe())

count                                    42985
unique                                   42985
top       993bbbd6-4dbc-4a40-a408-f91f8462bee6
freq                                         1
Name: id_, dtype: object


In [1479]:
print(df['id_'].value_counts(dropna=False))

id_
993bbbd6-4dbc-4a40-a408-f91f8462bee6    1
089b39d8-e4d0-461b-87d4-814d71e0e079    1
e9d37224-cb6f-4942-98d7-46672963d097    1
3f9f49b5-e410-4948-bf6e-f9244f04918b    1
9991fafb-4447-451a-8be2-b0df6098d13e    1
                                       ..
54b1ea8a-a909-45d9-9562-775b64ac9c29    1
66a752e2-b2f1-440c-9a8f-cc3b10d74dd0    1
ca5c0d7a-8bbd-42b3-8351-1074c5be011a    1
8d700df5-0c33-4517-8cf8-92e1c92c9c12    1
ea6b7d04-9271-4c0a-a01f-07795d164aba    1
Name: count, Length: 42985, dtype: int64


Como no hay nulos, esta columna está lista para el análisis descriptivo. Quedarían así los valores de la columna `id_` que nos indica el identificador único para cada registro en el dataset.

## Guardamos los datos limpios

In [1480]:
df.to_csv('../data/output/bank-additional_limpio.csv', index=False)